Moving Average Crossover Strategy

In [1]:
import requests
import numpy as np
import pandas as pd

intrinio_api_key = "OmIzODUyMTg2MTdkYTU5NGM0M2UxYzA1M2IxZWQyZTZi"

def _stock_prices_dataset(ticker):

  res = requests.get(f"https://api-v2.intrinio.com/securities/{ticker}/prices?page_size=1000&api_key={intrinio_api_key}")
  stock_prices_dataset = pd.DataFrame(res.json().get("stock_prices"))
  stock_prices_dataset = stock_prices_dataset[["date", "adj_close"]][::-1].reset_index(drop=True)

  stock_prices_dataset["returns"] = np.log(stock_prices_dataset["adj_close"]).diff()
  stock_prices_dataset["returns"] = stock_prices_dataset["returns"].shift(-1)

  return stock_prices_dataset

In [ ]:
def _sma_stock_prices_dataset(stock_prices_dataset, long_sma = 200, short_sma = 50):

  stock_prices_dataset[f"{long_sma}_day_sma"] = stock_prices_dataset["adj_close"].rolling(long_sma).mean()
  stock_prices_dataset[f"{short_sma}_day_sma"] = stock_prices_dataset["adj_close"].rolling(short_sma).mean()
  stock_prices_dataset = stock_prices_dataset.dropna()
  stock_prices_dataset = stock_prices_dataset.reset_index(drop=True)

  return stock_prices_dataset